<a href="https://colab.research.google.com/github/AshKanishk/Extracting_TextFromImage/blob/master/Extracting_TextFromImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading and Importing Important Libraries

In [0]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

from google.colab.patches import cv2_imshow  

#Loading dataset through zip file through files.upload

In [0]:
from google.colab import files

uploaded = files.upload()

##files.upload returns a dictionary of the files which were uploaded. The 
##dictionary is keyed by the file name, the value is the data which was 
##uploaded.

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving Dataset.zip to Dataset.zip
User uploaded file "Dataset.zip" with length 17502841 bytes


In [0]:
!unzip Dataset.zip

Importing "pytesseract" for extracting text from image

In [0]:
import IPython.display as display
import pytesseract
import shutil
import random
try:
 from PIL import Image
except ImportError:
 import Image
from PIL import Image

Funtion for pre processing of image before passing it through OCR

In [0]:
def image_enhance(img) :
  correct_skew(img) #correcting skew angle
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converting to grayscale
  img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)  #scaling image so that small characters can be read
  kernel = np.ones((1, 1), np.uint8)
  img = cv2.dilate(img, kernel, iterations=1) #for dilation
  img = cv2.erode(img, kernel, iterations=1) #for erosion
  cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2) #for threshold
  return img

Funtion to convert text from image to string and then evaluating each word to extract useful information

In [0]:
from dateutil.parser import parse
def extractText(img) :
  result= pytesseract.image_to_string(img,lang="eng")
  result = result.split()
  i=0
  for word in result :
    if(str(word).isalnum() and len(word)==17 ):
      i=1
      print(result)
  if(i==0) : print("NA")

Function for detecting skew angle and then correcting it

In [0]:
from scipy.ndimage import interpolation as inter

def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return rotated

Accessing the images from the dataset through a loop

In [0]:
path = os.path.join(r"Dataset")  # create path to Dataset
for img in os.listdir(path):  # iterate over each image 
  image = cv2.imread(img)
  image = image_enhance(image)
  print(img, end=" ")
  extractText(image)

  